In [28]:
import pandas as pd
import numpy as np

In [29]:
holidays = pd.read_csv("holidays_events.csv")
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [30]:
# dropping holidays that are transferred and holidays that are
# listed as work days (see data)
# adding is_holiday column for later
holidays = holidays[holidays.transferred == False]
holidays = holidays[holidays.type != "Work Day"]
is_holiday = [True] * len(holidays)
holidays["is_holiday"] = is_holiday
holidays

,date,type,locale,locale_name,description,transferred,is_holiday
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,True
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,True
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,True
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,True
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,True
...,...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,True
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,True
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,True
348,2017-12-25,Holiday,National,Ecuador,Navidad,False,True


In [31]:
train = pd.read_csv("train.csv")
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [32]:
#merging the train data with the holiday_boolean
train_merge = train.merge(holidays, on="date", how="left")
train_merge = train_merge.fillna(value = False)
train_merge = train_merge.drop(columns=["type", "locale", "locale_name", "description", "transferred"])
train = train_merge

In [33]:
#converting to date to datetime type
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')
train

,id,date,store_nbr,family,sales,onpromotion,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,True
1,1,2013-01-01,1,BABY CARE,0.000,0,True
2,2,2013-01-01,1,BEAUTY,0.000,0,True
3,3,2013-01-01,1,BEVERAGES,0.000,0,True
4,4,2013-01-01,1,BOOKS,0.000,0,True
...,...,...,...,...,...,...,...
3052561,3000883,2017-08-15,9,POULTRY,438.133,0,True
3052562,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,True
3052563,3000885,2017-08-15,9,PRODUCE,2419.729,148,True
3052564,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,True


In [23]:
test = pd.read_csv("test.csv")
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [28]:
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object